In [5]:
import numpy as np
import pandas as pd
import torch
from model import DrBC
from utils import prepare_test, prepare_synthetic, preprocessing_data, validate
MODEL_SAVED_PATH = "saved_model/"

### Experiment

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
model200 = DrBC().to(device)
model200.load_state_dict(torch.load(f'{MODEL_SAVED_PATH}DrBC_G1000_N200_E10000.pth'))

model5000 = DrBC().to(device)
model5000.load_state_dict(torch.load(f'{MODEL_SAVED_PATH}DrBC_G1000_N5000_E5000.pth'))

scales = [5000, 10000, 20000]
ls_metrics = []
for scale in scales:
    print('-'*15, scale)
    g_list, dg_list, bc_list = prepare_synthetic(30, (scale, scale+1), parallel=True)
    for i in range(len(g_list)):
        test_X, test_y, test_edge_index = preprocessing_data([g_list[i]], [dg_list[i]], [bc_list[i]])
        t_data = [[test_X, test_y, test_edge_index]]
        _acc1, _acc5, _acc10, _kendall, _time = validate(model200, t_data)
        ls_metrics.append([scale, '200', i, _acc1, _acc5, _acc10, _kendall, _time])

        _acc1, _acc5, _acc10, _kendall, _time = validate(model5000, t_data)
        ls_metrics.append([scale, '5000', i, _acc1, _acc5, _acc10, _kendall, _time])

t_data = prepare_test('y')
_acc1, _acc5, _acc10, _kendall, _time = validate(model200, t_data) 
ls_metrics.append(['youtube', '200', 0, _acc1, _acc5, _acc10, _kendall, _time])

_acc1, _acc5, _acc10, _kendall, _time = validate(model5000, t_data) 
ls_metrics.append(['youtube', '5000', 0, _acc1, _acc5, _acc10, _kendall, _time])

--------------- 5000


[Generating new training graph]: 100%|██████████| 30/30 [06:28<00:00, 12.96s/it]


--------------- 10000


[Generating new training graph]: 100%|██████████| 30/30 [12:50<00:00, 25.67s/it]


--------------- 20000


[Generating new training graph]: 100%|██████████| 30/30 [40:48<00:00, 81.60s/it]
/home/littlemilk/miniconda3/envs/jupyterhub/lib/python3.9/site-packages/scipy/stats/_stats_py.py:5218: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


In [8]:
df = pd.DataFrame(ls_metrics, columns=['scale', 'model', 'test_graph_id', 'test_acc1', 'test_acc5', 'test_acc10', 'test_kendall', 'time'])
df.to_csv('test_scale_diff_result.csv', index=False)